In [6]:
import os

In [7]:
# Change working dir to where yolov3
os.chdir(r'C:\Users\zhiqian\Desktop\GitHub\bball\from_github\yolov3')
os.getcwd()

'C:\\Users\\zhiqian\\Desktop\\GitHub\\bball\\from_github\\yolov3'

In [8]:
import argparse
from sys import platform

from models import *  # set ONNX_EXPORT in models.py
from utils.datasets import *
from utils.utils import *

In [14]:
source = '../videotest2.mp4'
cfg = '../basketball/cfg/basketball.cfg'
weights = '../basketball/weights/best.pt'
names = '../basketball/data/basketball.names '
output = 'output'
conf_thres = 0.7
iou_thres = 0.3
view_img = True
classify = False

In [15]:
img_size = (320, 192)
out, source, weights = output, source, weights
webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')

# Initialize
device = torch_utils.select_device(device='cpu')
if os.path.exists(out):
    shutil.rmtree(out)  # delete output folder
os.makedirs(out)  # make new output folder

# Initialize model
model = Darknet(cfg, img_size)

Using CPU



In [16]:
# Load weights
attempt_download(weights)
if weights.endswith('.pt'):  # pytorch format
    model.load_state_dict(torch.load(weights, map_location=device)['model'])
else:  # darknet format
    load_darknet_weights(model, weights)

model.to(device).eval()

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (Conv2d): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (Conv2d): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      

In [17]:
# Set Dataloader
vid_path, vid_writer = None, None
if webcam:
    view_img = True
    torch.backends.cudnn.benchmark = True  # set True to speed up constant image size inference
    dataset = LoadStreams(source, img_size=img_size)
else:
    save_img = True
    dataset = LoadImages(source, img_size=img_size)

# Get names and colors
names = load_classes(names)
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

In [18]:
# Run inference
t0 = time.time()
for path, img, im0s, vid_cap in dataset:
    t = time.time()

    # Get detections
    img = torch.from_numpy(img).to(device)
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    pred = model(img)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    # Apply Classifier
    if classify:
        pred = apply_classifier(pred, modelc, img, im0s)

    # Process detections
    for i, det in enumerate(pred):  # detections per image

        if webcam:  # batch_size >= 1
            p, s, im0 = path[i], '%g: ' % i, im0s[i]
        else:
            p, s, im0 = path, '', im0s

        save_path = str(Path(out) / Path(p).name)
        s += '%gx%g ' % img.shape[2:]  # print string
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += '%g %ss, ' % (n, names[int(c)])  # add to string

            # Write results
            for *xyxy, conf, cls in det:
#                 if save_txt:  # Write to file
#                     with open(save_path + '.txt', 'a') as file:
#                         file.write(('%g ' * 6 + '\n') % (*xyxy, cls, conf))

                if save_img or view_img:  # Add bbox to image
                    label = '%s %.2f' % (names[int(cls)], conf)
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])
        # Print time (inference + NMS)
        print('%sDone. (%.3fs)' % (s, time.time() - t))
        
        
        # Stream results
        if view_img:
            cv2.imshow(p, im0)
            if cv2.waitKey(1) == ord('q'):  # q to quit
                raise StopIteration

video 1/1 (1/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 1 defenses, 1 runs, 1 stands, Done. (0.486s)
video 1/1 (2/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 1 walks, 1 runs, Done. (0.502s)
video 1/1 (3/621) ..\videotest2.mp4: 192x320 1 baskets, 1 walks, 1 stands, Done. (0.512s)
video 1/1 (4/621) ..\videotest2.mp4: 192x320 1 baskets, 1 walks, 1 stands, Done. (0.495s)
video 1/1 (5/621) ..\videotest2.mp4: 192x320 1 walks, 1 runs, 1 stands, Done. (0.507s)
video 1/1 (6/621) ..\videotest2.mp4: 192x320 2 defenses, 2 walks, 1 stands, Done. (0.498s)
video 1/1 (7/621) ..\videotest2.mp4: 192x320 1 defenses, 1 walks, 1 stands, Done. (0.504s)
video 1/1 (8/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 1 defenses, 2 walks, 1 runs, Done. (0.494s)
video 1/1 (9/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 1 defenses, 1 walks, Done. (0.497s)
video 1/1 (10/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 2 defenses, 2 walks, Done. (0.493s)
video 1/1 (11/621

video 1/1 (89/621) ..\videotest2.mp4: 192x320 1 stands, Done. (0.514s)
video 1/1 (90/621) ..\videotest2.mp4: 192x320 1 defenses, Done. (0.493s)
video 1/1 (91/621) ..\videotest2.mp4: 192x320 1 defenses, Done. (0.492s)
video 1/1 (92/621) ..\videotest2.mp4: 192x320 1 defenses, Done. (0.490s)
video 1/1 (93/621) ..\videotest2.mp4: 192x320 1 defenses, Done. (0.487s)
video 1/1 (94/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 2 defenses, 1 walks, Done. (0.479s)
video 1/1 (95/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 1 defenses, 2 walks, Done. (0.489s)
video 1/1 (96/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 2 walks, Done. (0.474s)
video 1/1 (97/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, Done. (0.467s)
video 1/1 (98/621) ..\videotest2.mp4: 192x320 1 baskets, 1 referees, 1 walks, Done. (0.494s)
video 1/1 (99/621) ..\videotest2.mp4: 192x320 1 referees, 1 walks, Done. (0.472s)
video 1/1 (100/621) ..\videotest2.mp4: 192x320 1 referees, 1 walks, Done. (

StopIteration: 